**Task 07: Querying RDF(s)**

In [2]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2021-2022/master/Assignment4/course_materials"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 4.3 MB/s 
     |████████████████████████████████| 41 kB 538 kB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [51]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
from rdflib.plugins.sparql import prepareQuery
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")

<Graph identifier=Nc6aaebfa376a49c2883b29b235f2023d (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [54]:
### With RDFLib
print("With RDFLib:")
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s)

### With SPARQL
print()
print("With SPARQL:")
q1 = """
PREFIX ns: <http://somewhere#>
PREFIX vcard: <http://www.w3.org/2001/vcard-rdf/3.0/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?subclasses
WHERE {
    ?subclasses rdfs:subClassOf ns:Person
}"""

# Visualize the results
for r in g.query(q1):
  print(r)

With RDFLib:
http://somewhere#Researcher

With SPARQL:
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [78]:
### With RDFLib
print("With RDFLib:")
# Search for individuals
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s)
# Search for subclasses of Person
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  # Individuals of the subclasses found
  for s2, p2, o2, in g.triples((None, RDF.type, s)):
    print(s2)

### With SPARQL
print()
print("With SPARQL:")
q2 = prepareQuery(
    """SELECT DISTINCT ?name
    WHERE {
      ?name rdf:type ?subclass .
      ?subclass rdfs:subClassOf* ns:Person
    }""",
    initNs = { "ns": ns, "rdf": RDF, "rdfs": RDFS})


# Visualize the results
for r in g.query(q2):
  print(r)

With RDFLib:
http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith

With SPARQL:
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [89]:
### With RDFLib
print("With RDFLib:")
# Search for individuals
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  # Properties and its values of the individuals
  for s2, p2, o2 in g.triples((s, None, None)):
    print(s2, p2, o2)
# Search for subclasses of Person
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  # Individuals of the subclasses found
  for s2, p2, o2 in g.triples((None, RDF.type, s)):
    # Properties and its values of the individuals
    for s3, p3, o3 in g.triples((s2, None, None)):
      print(s3, p3, o3)

### With SPARQL
print()
print("With SPARQL:")
q3 = prepareQuery(
    """SELECT DISTINCT ?name ?property ?value
    WHERE {
      ?name rdf:type ?subclass .
      ?subclass rdfs:subClassOf* ns:Person .
      ?name ?property ?value
    }""",
    initNs = { "ns": ns, "rdf": RDF, "rdfs": RDFS})

# Visualize the results
for r in g.query(q3):
  print(r)

With RDFLib:
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http: